### Data Collection

In [28]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import config

In [29]:
api_key = config.api_key

## Part-1 “Comedy” genre movies released in the year 2000 or later.

In [30]:
table = []

In [31]:
for i in range(15):
    response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&with_genres=35&primary_release_date.gte=2000&sort_by=popularity.desc&page='+ str(i+1))
    # print(response)
    movies_list = response.json()
    movies = movies_list['results']
    for j in range(len(movies)):
        row = []
        movie_id = movies[j]['id']
        movie_name = movies[j]['original_title']
        # print(movie_id, movie_name)
        table.append([movie_id, movie_name])

In [32]:
df = pd.DataFrame(table, columns=['movie-ID', 'movie-name'])
df

,movie-ID,movie-name
0,512195,Red Notice
1,585245,Clifford the Big Red Dog
2,876716,Ciao Alberto
3,654974,Home Sweet Home Alone
4,550988,Free Guy
...,...,...
295,570269,劇場版トリニティセブン -天空図書館と真紅の魔王-
296,19899,Couples Retreat
297,291805,Now You See Me 2
298,308531,Teenage Mutant Ninja Turtles: Out of the Shadows


In [33]:
df.to_csv('movie_ID_name.csv', header=False, index=False)

## Part-2 Similar Movies from Part-1

In [34]:
table2 = []
for i in range(len(df['movie-ID'])):
    response = requests.get('https://api.themoviedb.org/3/movie/'+ str(df['movie-ID'][i]) +'/similar?api_key='+ api_key +'&language=en-US&page=1')
    similar_movies = response.json()
    movies = similar_movies['results']
    for j in range(5):
        row = []
        try:
            similar_movie_id = movies[j]['id']
        except:
            break
        table2.append([df['movie-ID'][i], similar_movie_id])

In [35]:
df2 = pd.DataFrame(table2, columns=['movie-ID', 'similar-movie-ID'])
df2

,movie-ID,similar-movie-ID
0,512195,9350
1,512195,8836
2,512195,8838
3,512195,9600
4,512195,949
...,...,...
1495,350,2978
1496,350,1633
1497,350,1643
1498,350,1646


In [36]:
df2.to_csv('movie_ID_sim_movie_ID.csv', header=False, index=False)

### Remove duplicates

In [37]:
updated_df = pd.DataFrame()

In [38]:
k=0
for i in range(len(df2['movie-ID'])):
    try:
        movie_id = df2['movie-ID'][i]
        sim_id = df2['similar-movie-ID'][i]
        df3 = df2[df2['movie-ID']==sim_id]
        df3 = df3.reset_index(drop=True)
        for movie in list(df3['similar-movie-ID']):
            if movie_id == movie:
                if movie_id < sim_id:
                   df2 = df2.drop(df2[df2['similar-movie-ID']==movie].index[0])
                else:
                   df2 = df2.drop(i)
    except:
        pass

In [39]:
updated_df.to_csv('updated_movie_ID_sim_movie_ID.csv', index=False)